In [1]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from lxml import etree
import time

# get player name
def get_player_name():
    player_list = []
    player = sp.find_all(class_ = 'full-3fV3c9pF')
    for i in range(0, len(player), 2):
        player_list.append(player[i].text + ' ' + player[i+1].text)
    return player_list

# get player's position
def get_player_position():
    position_list = []
    position = sp.find_all(class_ = 'position-28TbwVOg')
    for i in position:
        position_list.append(i.text)
    return position_list

def get_column_name():
    click_standard_button()
    sp = soup(driver.page_source, 'lxml')
    
    data_column_name = ['PLAYER', 'POSITION', 'TEAM']
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
    temp = sp.find_all(class_ = 'bui-text cellheader selected-1vxxHvFg bui-text')
    for i in range(0, len(temp), 2):
        data_column_name.append(temp[i].text)

    click_expanded_button()
    sp = soup(driver.page_source, 'lxml')
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
        
    return data_column_name

# click standard button
def click_standard_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    #time.sleep(0.5)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    time.sleep(2)
    sp = soup(driver.page_source, 'lxml')

# click expanded button
def click_expanded_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    #time.sleep(0.5)    
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    time.sleep(2)
    sp = soup(driver.page_source, 'lxml')

# click next page button
def click_next_page_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[3]/div[2]/div/div/div[2]/button').click()
    #time.sleep(0.5)
    #driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[3]/div[2]/div/div/div[2]/button').click()
    time.sleep(2)
    sp = soup(driver.page_source, 'lxml')

# get standard data
def get_data():
    data = []
    
    click_standard_button()
    sp = soup(driver.page_source, 'lxml')
    standard_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        standard_data.append(i.text)
    
    click_expanded_button()
    sp = soup(driver.page_source, 'lxml')
    expanded_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        expanded_data.append(i.text)

    for i in range(len(standard_data)//17):
        data.extend(standard_data[i*17:i*17+17])
        data.extend(expanded_data[i*16+1:i*16+16])

    return data

In [2]:
urls = pd.read_csv('mlb_stats_urls_2022.csv', encoding='utf-8')

for year in range(2023, 2024):
    for link in urls['link']:
        df_list = []
        for page in range(1, 4):
            url = link + str(year) + '?page=' + str(page) + '&playerPool=ALL'
            driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')
            html = driver.get(url)
            sp = soup(driver.page_source, 'lxml')
            column_name = get_column_name()
            try:
                #while True:
                sp = soup(driver.page_source, 'lxml')
                print('page start')
                time.sleep(1)
                name = get_player_name()
                time.sleep(1)
                position = get_player_position()
                time.sleep(1)
                current_data = get_data()
                time.sleep(1)
                for i in range(len(name)):
                    row = []
                    row.append(name[i])
                    row.append(position[i])
                    row.extend(current_data[i*32:i*32+32])
                    df_list.append(row)
                time.sleep(1)
                #click_next_page_button()
            except:
                print('This is the last page :)')
        df = pd.DataFrame(df_list, columns=column_name)
        df.to_csv(f'mlb_stats/{year}/{link[26:-1]}_{year}.csv', index=False, encoding='utf-8')
        print(df)

C:\Users\user\AppData\Local\Temp\ipykernel_47360\1135733809.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
              PLAYER POSITION TEAM    G   AB    R    H  2B 3B  HR  ... GO/AO  \
0      Shohei Ohtani       DH  LAA  135  497  102  151  26  8  44  ...  1.00   
1         Mike Trout       CF  LAA   82  308   54   81  14  1  18  ...  0.62   
2      Brandon Drury       2B  LAA  125  485   61  127  30  3  26  ...  0.94   
3      Mickey Moniak       CF  LAA   85  311   35   87  21  2  14  ...  0.63   
4      Logan O'Hoppe        C  LAA   51  182   23   43   6  0  14  ...  0.80   
5       Luis Rengifo       2B  LAA  126  394   55  104  15  4  16  ...  1.02   
6        Taylor Ward       LF  LAA   97  356   60   90  18  0  14  ...  0.88   
7     Hunter Renfroe       RF  LAA  126  459   56  111  31  0  19  ...  0.86   
8   Michael Stefanic        X  LAA   25   62    5   18   2  1   0  ...  0.80   
9     Nolan Schanuel       1B  LAA   29  109   19   30   3  0   1  ...  1.31   
10          Jo Adell       RF  LAA   17   58    7   12   3  1   3  ...  1.10   
11     

C:\Users\user\AppData\Local\Temp\ipykernel_47360\1135733809.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
              PLAYER POSITION TEAM    G   AB    R    H  2B 3B  HR  ... GO/AO  \
0     Yordan Alvarez       DH  HOU  114  410   77  120  24  1  31  ...  0.73   
1        Jose Altuve       2B  HOU   90  360   76  112  21  2  17  ...  1.17   
2        Kyle Tucker       RF  HOU  157  574   97  163  37  5  29  ...  0.75   
3        Yainer Diaz        C  HOU  104  355   51  100  22  0  23  ...  1.15   
4     Chas McCormick       LF  HOU  115  403   59  110  17  2  22  ...  0.99   
5       Alex Bregman       3B  HOU  161  622  103  163  28  4  25  ...  0.63   
6   Michael Brantley       LF  HOU   15   54    7   15   2  0   2  ...  0.73   
7     Mauricio Dubon       2B  HOU  132  467   76  130  26  3  10  ...  1.13   
8        Jeremy Pena       SS  HOU  150  577   81  152  32  3  10  ...  1.50   
9         Jose Abreu       1B  HOU  141  540   62  128  23  1  18  ...  0.93   
10       Jake Meyers       CF  HOU  112  309   42   70  16  1  10  ...  0.83   
11     

C:\Users\user\AppData\Local\Temp\ipykernel_47360\1135733809.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


...

page start
page start
page start
               PLAYER POSITION TEAM    G   AB   R    H  2B 3B  HR  ... GO/AO  \
0         Royce Lewis       3B  MIN   58  217  36   67   7  0  15  ...  0.70   
1        Matt Wallner       LF  MIN   76  213  42   53  11  1  14  ...  0.63   
2        Ryan Jeffers        C  MIN   96  286  46   79  15  2  14  ...  0.71   
3      Edouard Julien       2B  MIN  109  338  60   89  16  1  16  ...  2.13   
4          Max Kepler       RF  MIN  130  438  72  114  22  2  24  ...  0.72   
5      Alex Kirilloff       1B  MIN   88  281  35   76  14  1  11  ...  0.94   
6       Jorge Polanco       2B  MIN   80  302  38   77  18  0  14  ...  0.47   
7      Donovan Solano       1B  MIN  134  394  43  111  26  1   5  ...  0.88   
8        Willi Castro       LF  MIN  124  358  60   92  18  5   9  ...  0.79   
9          Joey Gallo       1B  MIN  111  282  39   50   9  1  21  ...  0.55   
10       Byron Buxton       DH  MIN   85  304  49   63  17  1  17  ...  0.56   
11     

C:\Users\user\AppData\Local\Temp\ipykernel_47360\1135733809.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
               PLAYER POSITION TEAM    G   AB   R    H  2B 3B  HR  ... GO/AO  \
0         Luis Robert       CF  CWS  145  546  90  144  36  1  38  ...  0.77   
1        Eloy Jimenez       DH  CWS  120  456  50  124  23  0  18  ...  1.48   
2       Andrew Vaughn       1B  CWS  152  566  67  146  30  2  21  ...  0.99   
3        Yoan Moncada       3B  CWS   92  334  39   87  20  1  11  ...  1.14   
4   Andrew Benintendi       LF  CWS  151  562  72  147  34  2   5  ...  0.82   
5        Elvis Andrus       2B  CWS  112  374  39   94  20  1   6  ...  1.44   
6      Hanser Alberto       3B  CWS   30   82  11   18   5  0   3  ...  1.08   
7     Yasmani Grandal        C  CWS  118  363  33   85  14  0   8  ...  0.98   
8      Zach Remillard       2B  CWS   54  147  16   37   7  0   1  ...  1.20   
9        Carlos Perez        C  CWS   27   49   5   10   4  0   1  ...  1.55   
10       Gavin Sheets       RF  CWS  118  311  24   63  10  0  10  ...  0.68   
11     

C:\Users\user\AppData\Local\Temp\ipykernel_47360\1135733809.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
                PLAYER POSITION TEAM    G   AB   R    H  2B 3B  HR  ... GO/AO  \
0          Aaron Judge       RF  NYY  106  367  79   98  16  0  37  ...  0.61   
1     Jasson Dominguez       CF  NYY    8   31   6    8   1  0   4  ...  2.00   
2           Greg Allen        X  NYY   22   23   6    5   1  1   1  ...  1.25   
3       Gleyber Torres       2B  NYY  158  596  90  163  28  2  25  ...  0.75   
4         Austin Wells        C  NYY   19   70   8   16   6  0   4  ...  1.28   
5       Billy McKinney       LF  NYY   48  128  19   29   3  1   6  ...  0.97   
6          DJ LeMahieu       3B  NYY  136  497  55  121  22  3  15  ...  1.84   
7       Willie Calhoun       DH  NYY   44  134  16   32   7  0   5  ...  0.69   
8        Anthony Rizzo       1B  NYY   99  373  45   91  14  0  12  ...  0.68   
9    Giancarlo Stanton       DH  NYY  101  371  43   71  13  0  24  ...  0.99   
10         Jake Bauers       RF  NYY   84  242  28   49  15  0  12  ...  0.9

In [ ]:
driver.quit()